In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode
from plotly.offline import iplot as iplot_offline
from plotly.offline import plot as plot_offline
from plotly.plotly import iplot, plot
from plotly.graph_objs import Scatter
# from plotly.tools import FigureFactory as FF
init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook

%matplotlib inline
import matplotlib as plt
plt.style.use('ggplot')
import scripts.daily_data as dd
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import graph_objs

import requests

In [34]:
print(__version__)

2.5.1


In [2]:
TBP_colors = {
    'red_orange': 'rgb(255, 64, 0)',
    'gold': 'rgb(247,195,24)',
    'orange': 'rgba(236,159,76,1)',
    'orange_sc': 'rgba(236,159,76,0.8)',
    'dark_orange': '#e8800f',
    'gray': '#e6e6e6',
    'light gray': '#f2f2f2',
    'white': 'rgb(255,255,255)',
    'teal': '#b2e3dd',
    'black opaque': 'rbga(0,0,0,0.65)',
    'purple': 'rgb(105,107,165)',
    'red': 'rgb(210,20,4)'
}

In [13]:

def convert_legacy_date(date):
    return np.datetime64(datetime.utcfromtimestamp(date.tolist()/1e9).date())

def convert_legacy_title(title):
    return title.replace(' —', ':')

def convert_since_title(title):
    return title.replace('?"', ':')

# Create Legacy Stats DataFrame
episodes_legacy = pd.read_csv('./raw_data/stats_TBP_episodic_legacy.csv', 
                                index_col=None, parse_dates=['release_date'])
episodes_legacy = episodes_legacy[['release_date', 'item_title', 'downloads__total']]
episodes_legacy.columns = ['release_date', 'title', 'downloads']
episodes_legacy['release_date'] = [convert_legacy_date(date) for date in episodes_legacy['release_date'].values]

# Create New Stats DataFrame
episodes_since = pd.read_csv('./raw_data/stats_TBP_episodic_2018-04-03.csv',
                                      index_col=None, parse_dates=['Released'])
episodes_since = episodes_since[['Released', 'Title', 'Downloads']]
episodes_since.columns = ['release_date', 'title', 'downloads']
episodes_since['release_date'] = [convert_legacy_date(date) for date in episodes_since['release_date'].values]

# Concatenate appropriately
episodes_legacy['title'] = [convert_legacy_title(title) for title in episodes_legacy['title']]
episodes_since['title'] = [convert_since_title(title) for title in episodes_since['title']]

# episodes_legacy = episodes_legacy.set_index(['release_date', 'title'])
# episodes_since = episodes_since.set_index(['release_date', 'title'])
episodes = pd.merge(episodes_legacy, episodes_since, how='outer', on=['title', 'release_date']).sort_values(by='release_date')
episodes['downloads_x'] = episodes.downloads_x.fillna(0)
episodes['downloads_y'] = episodes.downloads_y.fillna(0)
episodes['downloads'] = episodes.downloads_x + episodes.downloads_y
# episodes = pd.concat([episodes_legacy,episodes_since]).reset_index().sort_values(by='release_date')
episodes = episodes[episodes.title != "BTCPODcast.mp3"]

# # Get Days of Week DF
release_dates = list(episodes.release_date)

# days_of_week = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
# df_wkdays = daily
# df_wkdays['weekday'] = df_wkdays.reset_index().date.dt.dayofweek.values
# df_wkdays = df_wkdays.reset_index().pivot(index='date', columns='weekday', values='total_downloads')
# df_wkdays.columns = days_of_week.values()

# Adding show differentiation
def get_show(title):
    if "BC" in title:
        return 'BlockChannel'
    elif 'On-Ramping' in title:
        return 'On-Ramping with Dee'
    elif 'Announcements' in title:
        return 'TBP Announcements'
    elif 'NABP' in title:
        return 'Not Another Bitcoin Podcast'
    elif 'Ether Review' in title:
        return "The Ether Review"
    elif 'Buy or Sell' in title:
        return "Buy or Sell, What the Hell"
    elif 'An Ethereum' in title:
        return "An Ethereum Podcast"
    elif 'Humanist' in title:
        return 'Creating a Humanist Blockchain Future'
    else:
        return 'The Bitcoin Podcast'
episodes['show'] = episodes.title.apply(get_show)

# Let's add color differentiation
def get_color(title):
    if "BC" in title:
        return 'rgb(101,174,223)'
    elif 'TBP' in title:
        return TBP_colors['orange']
    elif 'On-Ramping' in title:
        return 'rgb(130,170,52)'
    elif 'Announcements' in title:
        return 'rgb(152,118,84)'
    elif "Ether Review" in title:
        return TBP_colors['black opaque']
    elif "NABP" in title:
        return TBP_colors['white']
    elif "Buy or Sell" in title:
        return TBP_colors['red']
    elif "An Ethereum" in title:
        return TBP_colors['purple']
    elif "Humanist" in title:
        return TBP_colors['red']
    else:
        return TBP_colors['orange']
episodes['color'] = episodes.title.apply(get_color)

In [49]:
bos_eps = episodes[episodes.show == "Buy or Sell, What the Hell"]#.downloads.plot(kind='bar')
tbp_eps = episodes[episodes.show == "The Bitcoin Podcast"]
eth_eps = episodes[episodes.show == "An Ethereum Podcast"]
tbp_eps.head()

,release_date,title,downloads_x,downloads_y,downloads,show,color
283,2015-06-01,Episode #1 - Introduction,3160.0,38.0,3198.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
282,2015-06-07,TBP02 - Holding Up the Sky,2546.0,14.0,2560.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
281,2015-06-15,TBP03 - Bridging the Gender Gap,2391.0,11.0,2402.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
280,2015-06-20,TBP04 - Change Jar 2.0,2510.0,10.0,2520.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
279,2015-06-28,TBP05 - The World's Strongest,2387.0,10.0,2397.0,The Bitcoin Podcast,"rgba(236,159,76,1)"


In [50]:
tbp_episodes = go.Bar(
    name = "The Bitcoin Podcast",
    x = tbp_eps['release_date'],
    y = tbp_eps.downloads,
    marker=dict(
        color=tbp_eps.color,
        line=dict(
            color='#e8800f',
            width=0,
        )
    ),
    opacity=0.85,
)

bos_episodes = go.Bar(
    name = "Buy or Sell",
    x = bos_eps['release_date'],
    y = bos_eps.downloads,
    marker=dict(
        color=bos_eps.color,
        line=dict(
            color='#e8800f',
            width=0,
        )
    ),
    opacity=0.85,
)

eth_episodes = go.Bar(
    name = "An Ethereum Podcast",
    x = eth_eps['release_date'],
    y = eth_eps.downloads,
    marker=dict(
        color=eth_eps.color,
        line=dict(
            color='#e8800f',
            width=0,
        )
    ),
    opacity=0.85,
)

# barh_sc = go.Bar(
#     name="SoundCloud",
#     x = sc_stats.playback_count,
#     y = sc_stats.title,
#     marker=dict(
#         color=sc_stats.color,
#         line=dict(
#             color='#e8800f',
#             width=0,
#         )
#     ),
#     opacity=0.85,
#     orientation = 'h'
# )

layout = go.Layout(
#     barmode="stack",
    title='Downloads by Episode',# + update_string,
#     margin=go.Margin(
#         l=375,
#         r=30,
#         b=30,
#         t=50,
#         pad=4
#     ),
    yaxis = dict(
#         title = 'Total downloads',
        gridcolor='#f2f2f2',
#         zerolinecolor='#969696',
#         showline=True,
    ),
    xaxis = dict(
#         zerolinecolor='#969696'
        gridcolor='#f2f2f2',
#         tickangle=45,
    ),
#     paper_bgcolor='rgba(242,242,242,0)',
    plot_bgcolor='#d7d8d8',
    font=dict(family='Helvetica, sans-serif', size=16, color='#2e2e2e'),
    width=1200,
    legend=dict(
        x=0.8,
        y=1
    )
)
# fig = dict(data=[barh_episodes, barh_sc], layout=layout)
fig = dict(data=[tbp_episodes, bos_episodes, eth_episodes], layout=layout)
iplot_offline(fig)
# iplot(fig, filename='TBP_plots/Downloads by Episode')

In [38]:
tbp_eps[tbp_eps.release_date > pd.Timestamp(2017,8,6)]

,release_date,title,downloads_x,downloads_y,downloads,show,color
84,2017-08-09,TBP145 - Hopes and Fears for Filecoin,6702.0,166.0,6868.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
83,2017-08-13,TBP146 - The Rise and Rise of Charlie Shrem,8174.0,145.0,8319.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
81,2017-08-16,TBP147 - History of Security,4893.0,158.0,5051.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
80,2017-08-17,BlockChannel Episode 27: Po.et and We Didn't E...,4555.0,165.0,4720.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
79,2017-08-17,BlockChannel Episode 28: Securing the Metavers...,4669.0,168.0,4837.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
78,2017-08-20,TBP148 - Controversial Persona,5776.0,174.0,5950.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
75,2017-08-27,TBP149 - The Rivetz Security Project,5337.0,155.0,5492.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
71,2017-09-01,BlockChannel Episode 29: Making Sense of Cent.co,4910.0,183.0,5093.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
70,2017-09-04,TBP150 - EGO with ETH,5236.0,194.0,5430.0,The Bitcoin Podcast,"rgba(236,159,76,1)"
67,2017-09-07,TBP151 - Crypto Boom Boom Room,4844.0,219.0,5063.0,The Bitcoin Podcast,"rgba(236,159,76,1)"


In [51]:
tbp_eps = tbp_eps[tbp_eps.release_date > pd.Timestamp(2017,8,1)]
averages = []
for x in range(len(tbp_eps)):
    average = tbp_eps.reset_index().loc[:x, 'downloads'].cumsum().values[-1] / (x+1)
    averages.append(average)
trace = go.Scatter(
    x = tbp_eps.set_index('release_date').index,
    y = averages,
    name = "TBP average",
    line = dict(
        color=TBP_colors['dark_orange']
    ),
    text=tbp_eps.title
)

averages = []
for x in range(len(bos_eps)):
    average = bos_eps.reset_index().loc[:x, 'downloads'].cumsum().values[-1] / (x+1)
    averages.append(average)
trace_bos = go.Scatter(
    x = bos_eps.set_index('release_date').index,
    y = averages,
    name = "BoS average",
    line = dict(
        color=TBP_colors['red']
    ),
    text=bos_eps.title
)

averages = []
for x in range(len(tbp_eps)):
    average = eth_eps.reset_index().loc[:x, 'downloads'].cumsum().values[-1] / (x+1)
    averages.append(average)
trace_eth = go.Scatter(
    x = eth_eps.set_index('release_date').index,
    y = averages,
    name = "AEP average",
    line = dict(
        color=TBP_colors['purple']
    ),
    text=eth_eps.title
)

rolling = go.Scatter(
    x = tbp_eps.set_index('release_date').index[4:],
    y = tbp_eps.set_index('release_date')['downloads'].rolling(10).mean(),
    name = 'Rolling Ave. (10 episodes)',
    line = dict(
        color=TBP_colors['orange']
    ),
)

rolling_bos = go.Scatter(
    x = bos_eps.set_index('release_date').index[4:],
    y = bos_eps.set_index('release_date')['downloads'].rolling(10).mean(),
    name = 'Rolling Ave. (10 episodes)',
    line = dict(
        color=TBP_colors['red']
    ),
)
layout = {
    'title': "Average Downloads per Episode",# + update_string,
    'xaxis': {
        'title': 'Episode number',
        'gridcolor': '#f2f2f2',
        'zeroline': False,
    },
    'yaxis': {
        'title': 'Average downloads',
        'gridcolor': '#f2f2f2',
        'zeroline': False,
    },
    'paper_bgcolor': 'rgba(242,242,242,0)',
    'plot_bgcolor': '#d7d8d8',
    'font': dict(family='Helvetica, sans-serif', size=16, color='#2e2e2e'),
    'showlegend':True,
}
fig = dict(data=[trace, trace_bos, trace_eth], layout=layout)
iplot_offline(fig)
#plot(fig, filename='TBP_plots/average-downloads')